In [ ]:
from magic_values import *

In [ ]:
with open("test_file", "rb") as f:
    data = f.read()
    
import struct

new_data = bytes(data)
print(MAGIC_MARKER)

while True:
    offset = new_data.find(MAGIC_MARKER)
    if offset < 0:
        break
    
    read_offset = offset + len(MAGIC_MARKER)
    print(new_data[read_offset:][:4])
    count = struct.unpack("I", new_data[read_offset:][:4])[0]
    read_offset += 4
    
    print(f"Found at offset: {hex(offset)}  count: {count}")
    
    exponents = new_data[read_offset:read_offset+count]
    read_offset += count
    mantissas = new_data[read_offset:read_offset+3*count]
    read_offset += 3 * count
    
    exponents = struct.unpack(f"{count}B", exponents)
    mantissas = struct.unpack(f"{3*count}B", mantissas)
    
    floats = [
        (exponents[i] << 23) 
            | (mantissas[i*3 + 2])
            | (mantissas[i*3 + 1] << 8)
            | ((mantissas[i*3 + 0] << 16) & 0x007F0000)
            | ((mantissas[i*3 + 0] << 24) & 0x80000000)
        for i in range(count)
    ]
    floats = struct.pack(f"{count}I", *floats)
    
    new_data = new_data[:offset] + floats + new_data[read_offset:]


In [ ]:
with open("reconstructed", "wb") as f:
    f.write(new_data)

In [ ]:
offset